## Notebook imports
- ipynb import documentation: https://ipynb.readthedocs.io/en/latest/, https://github.com/ipython/ipynb

In [10]:
from ipynb.fs.full.wordle_functions import wordle_wizard # importing all functions from wordle_functions nb
import urllib.request # for downloading wordle words from source
import numpy as np # for stats
import random # for randomly generating target and start words
import operator # for sorting letter frequency distribution
import time # for #dramaticeffect
import pandas as pd
from nltk.corpus import movie_reviews, treebank, brown, gutenberg, switchboard

## Importing datasets

### official words

In [2]:
### Official list
official_words = []

with open("data/official_words_processed.txt", "r", encoding = "utf-8") as f:
    for word in f.read().split("\n"):
        official_words.append(word)

f.close() # closes connection to file

print(len(official_words))
official_words[:5]

2310


['comfy', 'elbow', 'folio', 'audit', 'blast']

### alternative list 1

In [3]:
### Official list
alt_words_1 = []

with open("data/alt_words_1.txt", "r", encoding = "utf-8") as f:
    for word in f.read().split("\n"):
        alt_words_1.append(word)

f.close() # closes connection to file

print(len(alt_words_1))
alt_words_1[:5]

14856


['rossa', 'jetty', 'wizzo', 'cuppa', 'cohoe']

### nltk grand corpus
- Amalgamation of all words in various NLTK corpora to have as big a dataset as possible

In [4]:
### grand corpus tokens
nltk_tokens = []

with open("data/nltk_grand_corpus_tokens.txt", "r", encoding = "utf-8") as f:
    for word in f.read().split("\n"):
        nltk_tokens.append(word)

f.close() # closes connection to file

print(len(nltk_tokens))
nltk_tokens[:5]

535189


['years', 'board', 'dutch', 'group', 'agnew']

### nltk grand corpus types and counts

In [5]:
### grand corpus types and counts
nltk_counts = {}

with open("data/nltk_grand_corpus_types_and_counts.txt", "r", encoding = "utf-8") as f:
    for line in f.read().split("\n"):
        if len(line.split("\t")) == 2:
            word = line.split("\t")[0]
            count = line.split("\t")[1]
            nltk_counts[word] = count
        else:
            continue

f.close() # closes connection to file

print(len(nltk_counts))
nltk_counts['which']

8043


'15760'

## Simluations

In [6]:
# ## 13:45min for 1 guess word, all target words, 3 biases

# excepts = []

# for word in official_words:

#     for bias_option in ["entropy", "rare", "common", "no_bias"]:

#         try:
#             complete = wordle_wizard(word_list = official_words, max_guesses = 15, 
#                 guess = "later", target = word, bias = bias_option,
#                 random_guess = False, random_target = False, 
#                 verbose = False, drama = 0, return_stats = True)
            
#         except:
#             IndexError
#             excepts.append((complete["first_guess"], complete["target_word"]))

#         for metric, result in complete.items():
#             if metric in stats_dict.keys():
#                 stats_master[metric].append(result)
#             else:
#                 stats_master[metric] = []
                
# sims_df = pd.DataFrame(stats_master)
# # sims_df

In [7]:
excepts

NameError: name 'excepts' is not defined

In [ ]:
sims_df_valid = sims_df.query("valid_success == True")
# sims_df_valid

In [ ]:
# sims_df = pd.DataFrame(stats_master)
print(sims_df.value_counts("target_guessed", normalize = True))
print("-----")
print(sims_df.value_counts("valid_success", normalize = True))
print("-----")
print(sims_df.value_counts("num_guesses", normalize = True))
sims_df.sort_values(by = "num_guesses", ascending = False)

In [ ]:
print(sims_df.query("bias == 'entropy'")['num_guesses'].mean())
print(sims_df.query("bias == 'rare'")['num_guesses'].mean())
print(sims_df.query("bias == 'common'")['num_guesses'].mean())
print(sims_df.query("bias == 'no_bias'")['num_guesses'].mean())

In [ ]:
# arose_df = sims_df.query("first_guess == 'arose' & valid_success == True & bias == 'entropy'")
# print(round(arose_df['num_guesses'].mean(), 2))
# arose_df

In [ ]:
# arose_df = sims_df.query("first_guess == 'arose' & valid_success == True & bias == None")
# print(round(adore_df['num_guesses'].mean(), 2))
# adore_df

In [ ]:
# opera_df = sims_df.query("first_guess == 'arose' & valid_success == True & bias == 'entropy'")
# print(round(opera_df['num_guesses'].mean(), 2))
# opera_df

In [ ]:
# opera_df = sims_df.query("first_guess == 'opera' & valid_success == True & bias == None")
# print(round(opera_df['num_guesses'].mean(), 2))
# opera_df